##### *Reminder - use daily log to track progress

In [2]:
#!pip install librosa

In [5]:
import pandas as pd
import numpy as np
import os, pathlib
from sklearn.model_selection import train_test_split
import librosa
import torch
import IPython.display as ipd
from random import randint

In [6]:
#!sudo apt-get install wget unzip

In [7]:
# Download training data
#!mkdir -p data && cd data && wget https://irmas-dataset.s3-eu-west-1.amazonaws.com/IRMAS-TrainingData.zip && unzip IRMAS-TrainingData.zip 

In [8]:
#Download testing data (Around 2GB each)
#wget https://irmas-dataset.s3-eu-west-1.amazonaws.com/IRMAS-TestingData-Part1.zip && IRMAS-TestingData-Part1.zip
#wget https://irmas-dataset.s3-eu-west-1.amazonaws.com/IRMAS-TestingData-Part2.zip && IRMAS-TestingData-Part2.zip
#wget https://irmas-dataset.s3-eu-west-1.amazonaws.com/IRMAS-TestingData-Part3.zip && IRMAS-TestingData-Part3.zip    

IRMAS dataset (training)
 
Audio files: 6705 audio files in 16 bit stereo wav format sampled at 44.1kHz. They are excerpts of 3 seconds from more than 2000 distinct recordings. 

Annotations: The annotation of the predominant instrument of each excerpt is both in the name of the containing folder, and in the file name: cello (cel), clarinet (cla), flute (flu), acoustic guitar (gac), electric guitar (gel), organ (org), piano (pia), saxophone (sax), trumpet (tru), violin (vio), and human singing voice (voi). The number of files per instrument are: cel(388), cla(505), flu(451), gac(637), gel(760), org(682), pia(721), sax(626), tru(577), vio(580), voi(778). 

Additionally, some of the files have annotations in the filename regarding the presence ([dru]) or non presence([nod]) of drums, and the musical genre: country-folk ([cou_fol]), classical ([cla]), pop-rock ([pop-roc]), latin-soul ([lat-sou]).

In [33]:
IRMAS_TRAINING = 'data/IRMAS-TrainingData'
base_path = pathlib.Path(IRMAS_TRAINING)
classes, paths = [], []

for p in base_path.glob('*/*'):
    relative_path = p.relative_to(base_path)
    classes.append(str(relative_path.parent))
    paths.append(p)
    
#classes = set(classes)
df = pd.DataFrame({"tags": classes, "wav_path": paths}).sample(frac=1)#.reset_index(drop=True)#, inplace=True)
#df_training.piano = df.tags.map({'pia': True, '': 0})
#df.tags = df.tags.map({'pia': True, })
df.reset_index(drop=True, inplace=True)
df.head(5)

,tags,wav_path
0,pia,data/IRMAS-TrainingData/pia/[pia][cla]1327__1.wav
1,org,data/IRMAS-TrainingData/org/[org][pop_roc]1050...
2,pia,data/IRMAS-TrainingData/pia/029__[pia][nod][cl...
3,gac,data/IRMAS-TrainingData/gac/040__[gac][nod][co...
4,gel,data/IRMAS-TrainingData/gel/184__[gel][dru][po...


In [34]:
df.shape

(6705, 2)

In [35]:
classes = set(classes)

In [36]:
classes

{'cel', 'cla', 'flu', 'gac', 'gel', 'org', 'pia', 'sax', 'tru', 'vio', 'voi'}

In [37]:
amount_of_labels = len(classes)

In [38]:
classes = list(classes)

In [39]:
type(classes)

list

In [40]:
#inst_dict = {1:'cel', 2:'cla', 3:'flu', 4:'gac', 5:'gel',6:'org',7:'pia',8:'sax',9:'tru',10:'vio',11:'voi'}

In [41]:
class_dict = { i: classes[i] for i in range(0, len(classes))}

In [42]:
class_dict

{0: 'tru',
 1: 'cel',
 2: 'pia',
 3: 'voi',
 4: 'vio',
 5: 'cla',
 6: 'gel',
 7: 'gac',
 8: 'org',
 9: 'flu',
 10: 'sax'}

In [43]:
class_dict[9]

'flu'

In [44]:
df_train, df_test = train_test_split(df, test_size=0.3)

In [45]:
df_train.shape

(4693, 2)

In [46]:
df_test.shape

(2012, 2)

In [47]:
# Test if files are loading
successful, corrupted = [], []
len_x, sample_rates = [], []
raw_samples, sample_rates = [], []
for p in df.wav_path:
    try:
        x, sr = librosa.load(p, sr=None)
        successful.append(p)
        len_x.append(len(x))
        raw_samples.append(x)
        sample_rates.append(sr)
    except:
        corrupted.append(p)
#       print(p)
###df = df[~df.wav_path.isin(corrupted)]
assert len(successful) == len(raw_samples)
len(successful), len(corrupted), len(raw_samples), len(sample_rates)

(6705, 0, 6705, 6705)

In [48]:
set(len_x), set(sample_rates), 132299/44100

({132299}, {44100}, 2.9999773242630385)

In [49]:
df_raw = df.copy()

In [50]:
df_raw['raw_sounds'] = raw_samples
df_raw['sample_rate'] = sample_rates
df_raw

,tags,wav_path,raw_sounds,sample_rate
0,pia,data/IRMAS-TrainingData/pia/[pia][cla]1327__1.wav,"[-0.18333435, -0.17762756, -0.17004395, -0.160...",44100
1,org,data/IRMAS-TrainingData/org/[org][pop_roc]1050...,"[0.077819824, 0.015899658, -0.010467529, 0.004...",44100
2,pia,data/IRMAS-TrainingData/pia/029__[pia][nod][cl...,"[0.0066833496, 0.006088257, 0.005340576, 0.004...",44100
3,gac,data/IRMAS-TrainingData/gac/040__[gac][nod][co...,"[-0.012786865, -0.014755249, -0.011047363, -0....",44100
4,gel,data/IRMAS-TrainingData/gel/184__[gel][dru][po...,"[-0.08392334, -0.08354187, -0.0821991, -0.0809...",44100
...,...,...,...,...
6700,gel,data/IRMAS-TrainingData/gel/[gel][pop_roc]0912...,"[0.020599365, -0.01008606, -0.055541992, -0.09...",44100
6701,org,data/IRMAS-TrainingData/org/[org][jaz_blu]1205...,"[0.014587402, 0.009048462, 0.009521484, 0.0173...",44100
6702,cla,data/IRMAS-TrainingData/cla/[cla][pop_roc]0202...,"[0.06440735, 0.10116577, 0.14259338, 0.1802215...",44100
6703,gac,data/IRMAS-TrainingData/gac/082__[gac][nod][co...,"[0.049957275, 0.07394409, 0.09677124, 0.127120...",44100


In [51]:
import json

#with open('Data/IRMAS_raw.json', 'w') as f:
#    json.dumps(df.T.to_dict(orient='list'), f)
#df_raw.to_json('Data/IRMAS_raw.json')
#df_raw.to_json('Data/IRMAS_raw.json')

In [52]:
class InstrumentClassificationDataset():
    def __init__(self, df):
        super().__init__()
        self.df = df.copy()
#        self.image_transform = image_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        sound = self.load_from_disk(index)
        label = self.load_label(index)
#        Xi = self.image_transform(Xi)
        return sound, label

    def load_to_librosa(self, path):
        image = librosa.load(path)
        return image

    def load_from_disk(self, index):
        wav_path = df.iloc[index].wav_path 
        return self.load_to_librosa(wav_path)

    def load_label(self, index):
        label = df.iloc[index].tags
        return label

In [53]:
class BaseSampler():
    def __init__(self, df, list_of_instruments, n_samples):
        self.df = df.copy()
        self.n_samples = n_samples
        self.instruments = list_of_instruments
        #df = df[self.instruments]
        df = df[df['tags'].isin(self.instruments)]
        
    def __iter__(self):
        return iter(self._get_sample())
        
    def __len__(self):
        return self.n_samples
    
    def _get_samples(self):
        return np.random.choice(len(self.df), self.n_samples, replace=False)

In [54]:
# define the instruments we are using
list_of_instruments = ['sax', 'pia']

In [55]:
bs = BaseSampler(df, list_of_instruments, 30)

In [56]:
bs._get_samples()

array([1788, 1843, 2640, 5572, 1312, 4983, 1880, 4296, 5373,  809,  248,
       4917, 6326, 6346,  315, 6347, 5204, 4479, 4232,  744, 1861, 4700,
       2559,   21,  685, 5282, 4082, 6358, 4577, 2876])

In [57]:
batch_index = bs._get_samples()

In [58]:
ds_train = InstrumentClassificationDataset(df_train)

In [155]:
def sound_generator(df, inst, batch_size = 30):
    df = df[df['tags'].isin(inst)].copy()
    while True:
        batch_x, labels = [], []
        batch_index = np.random.choice(len(df), batch_size, replace=False)
        for idx in batch_index:
            wav_path = df.iloc[idx].wav_path 
            sound_frame, sr = librosa.load(wav_path, sr=None)
            batch_x.append(sound_frame)
            labels.append(df.iloc[idx].tags)
#        batch_x = np.array( batch_x )
#        labels = np.array (labels)
#        batch_x = np.expand_dims(sounds, axis=2)
        yield( batch_x, labels )

In [156]:
sg  = sound_generator(df_train, list_of_instruments, 30)

In [157]:
tg = sound_generator(df_test, list_of_instruments, 30)

In [158]:
sounds, labels = next(sg)

In [159]:
sounds = np.array(sounds)

In [160]:
sounds.shape

(30, 132299)

In [120]:
sounds[1], labels[1]

(array([0.03346252, 0.02574158, 0.01747131, ..., 0.01159668, 0.00952148,
        0.0075531 ], dtype=float32),
 'pia')

In [121]:
len(sounds[1]), min(sounds[1]), max(sounds[1])

(132299, -0.19787598, 0.20223999)

In [122]:
# Check if the sounds and labels match 
random = randint(0,29)
print(labels[random])
ipd.Audio(sounds[random], rate=44100)

sax


In [146]:
sounds.shape

(30, 132299, 1)

In [147]:
sounds

array([[[-0.04385376],
        [-0.04049683],
        [-0.04602051],
        ...,
        [ 0.04766846],
        [ 0.05493164],
        [ 0.05545044]],

       [[ 0.03346252],
        [ 0.02574158],
        [ 0.01747131],
        ...,
        [ 0.01159668],
        [ 0.00952148],
        [ 0.0075531 ]],

       [[-0.00683594],
        [-0.00657654],
        [-0.00627136],
        ...,
        [ 0.0055542 ],
        [ 0.00680542],
        [ 0.00326538]],

       ...,

       [[ 0.16339111],
        [ 0.1721344 ],
        [ 0.18226624],
        ...,
        [ 0.09150696],
        [ 0.08813477],
        [ 0.08370972]],

       [[ 0.05516052],
        [ 0.10282898],
        [ 0.13343811],
        ...,
        [ 0.11199951],
        [ 0.11482239],
        [ 0.11959839]],

       [[-0.25881958],
        [-0.28485107],
        [-0.30062866],
        ...,
        [ 0.05996704],
        [ 0.07139587],
        [ 0.07983398]]], dtype=float32)

In [135]:
X = np.expand_dims(sounds, axis=2)
X.shape

(30, 132299, 1)

In [136]:
X

array([[[-0.04385376],
        [-0.04049683],
        [-0.04602051],
        ...,
        [ 0.04766846],
        [ 0.05493164],
        [ 0.05545044]],

       [[ 0.03346252],
        [ 0.02574158],
        [ 0.01747131],
        ...,
        [ 0.01159668],
        [ 0.00952148],
        [ 0.0075531 ]],

       [[-0.00683594],
        [-0.00657654],
        [-0.00627136],
        ...,
        [ 0.0055542 ],
        [ 0.00680542],
        [ 0.00326538]],

       ...,

       [[ 0.16339111],
        [ 0.1721344 ],
        [ 0.18226624],
        ...,
        [ 0.09150696],
        [ 0.08813477],
        [ 0.08370972]],

       [[ 0.05516052],
        [ 0.10282898],
        [ 0.13343811],
        ...,
        [ 0.11199951],
        [ 0.11482239],
        [ 0.11959839]],

       [[-0.25881958],
        [-0.28485107],
        [-0.30062866],
        ...,
        [ 0.05996704],
        [ 0.07139587],
        [ 0.07983398]]], dtype=float32)

In [161]:
from tensorflow.keras import layers, models

model = models.Sequential()
#model.add(layers.Conv1D(1, kernel_size = 200, input_shape=(13299,1))) #
model.add(layers.Conv1D(filters = 64, kernel_size=200, activation='relu', input_shape=(132299,1,)))
model.add(layers.MaxPooling1D(pool_size=4))
#model.add(LSTM(64))
model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(amount_of_labels))
model.add(layers.Activation('softmax'))
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_16 (Conv1D)           (None, 132100, 64)        12864     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 33025, 64)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 2113600)           0         
_________________________________________________________________
dense_20 (Dense)             (None, 100)               211360100 
_________________________________________________________________
dense_21 (Dense)             (None, 11)                1111      
_________________________________________________________________
activation_10 (Activation)   (None, 11)                0         
Total params: 211,374,075
Trainable params: 211,374,075
Non-trainable params: 0
_______________________________________

In [162]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [163]:
#model.fit(trainX, trainy, epochs=10, verbose=verbose)

In [164]:
history = model.fit_generator(
    sg,
    steps_per_epoch=100,
    epochs=10,
    validation_data=tg,
    validation_steps=50
)

Epoch 1/10


ValueError: Error when checking input: expected conv1d_16_input to have 3 dimensions, but got array with shape (132299, 1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model.history.history["loss"], label="loss")
plt.plot(model.history.history["val_loss"], label="val_loss")
plt.legend()
plt.show()
plt.close()

plt.plot(model.history.history["accuracy"], label="accuracy")
plt.plot(model.history.history["val_accuracy"], label="val_accuracy")
plt.legend()
plt.show()
plt.close()